In [1]:
import pandas as pd
data = pd.read_csv("SMARD_Realisierter_Stromverbrauch_201811170000_202011122345_1.csv")

# data['Date'] = data['Date'].astype('str')
# data['Time'] = data['Time'].astype('str')
# data['Date'] = data['Date'] + ' '+ data['Time']

data = data.drop_duplicates(subset='Date', keep='first')
data.set_index('Date', inplace=True)

# data = data.assign(Date=data.Date.dt.round('H'))
# data = data.assign(Date=data.Date.dt.round('H'))
# data = data['01.01.2019':'01.07.2019']
data.head()

,Time,Demand[MWh]
Date,,
17.11.2018,00:00,118
18.11.2018,00:00,995
19.11.2018,00:00,985
20.11.2018,00:00,11425
21.11.2018,00:00,1265


In [2]:
data.index = pd.to_datetime(data.index)
data = data['Demand[MWh]']
data.columns = ['Energy Production']
# data = data[data.index.day == 1]
data = data.sort_index()

print(len(data))
# data.columns
# data = data['2019-01-01 00:00':'2019-06-01 00:00,']
# data = data['2019-01-01':]
data.tail()

365


Date
2019-12-07    125
2019-12-08     91
2019-12-09    106
2019-12-10    107
2019-12-11    121
Name: Demand[MWh], dtype: int64

In [3]:
data.columns = ['Energy Production']

train = data[:int((70*len(data)/100))]
valid = data[int((70*len(data)/100)):]

In [ ]:
import cufflinks as cf
import plotly.offline
from  plotly.offline import plot_mpl
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

data.iplot(title="power")

In [ ]:
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(data, lags=50)
pyplot.show()

In [ ]:

plot_pacf(data, lags=50)
pyplot.show()

In [ ]:
# decomposed = seasonal_decompose(data, model='additive')
# x = decomposed.plot() #See note below about this

In [ ]:
from pyramid.arima import auto_arima
model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)

# forecast = model.predict(n_periods=len(valid))
# forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

#plot the predictions for validation set

In [ ]:
future_forecast = model.predict(n_periods=len(valid))
future_forecast


In [ ]:
future_forecast = pd.DataFrame(future_forecast,index = valid.index,columns=['Prediction'])


In [ ]:
future_forecast = pd.DataFrame(future_forecast,index = valid.index,columns=['Prediction'])
pd.concat([valid,future_forecast],axis=1).iplot()

In [ ]:
pd.concat([data,future_forecast],axis=1).iplot()


In [ ]:
#calculate rmse
from math import sqrt
from sklearn.metrics import mean_squared_error

rms = sqrt(mean_squared_error(valid,future_forecast))
print(rms)